In [1]:
pip install prophet

Note: you may need to restart the kernel to use updated packages.


In [2]:
from prophet import Prophet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import datetime
import warnings

warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [30]:
df = pd.read_csv('crime_murder.csv', encoding='cp949', index_col=0)
df.head()

20년1월_살인  20년2월_살인  20년3월_살인  20년4월_살인  20년5월_살인  20년6월_살인  20년7월_살인  \
자치구                                                                         
강남구         0         0         1         0         0         0         1   
강동구         0         0         0         0         1         0         0   
강북구         0         1         0         0         1         3         1   
강서구         1         0         1         2         3         0         0   
관악구         1         1         0         1         0         2         1   

     20년8월_살인  20년9월_살인  20년10월_살인  ...  22년3월_살인  22년4월_살인  22년5월_살인  \
자치구                                 ...                                 
강남구         0         1          2  ...         1         0         2   
강동구         0         1          1  ...         0         0         0   
강북구         2         1          1  ...         0         1         0   
강서구         0         0          2  ...         0         0         0   
관악구         3         0          0  ...         0         1         2   

     22년6월_살인  22년7월_살인  22년8월_살인  22년9월_살인  22년10월_살인  22년11월_살인  22년12월_살인  
자치구                                                                           
강남구         3         0         1         3          1          1          0  
강동구         1         0         0         1          0          1          1  
강북구         0         2         0         2          0          0          0  
강서구         0         1         0         0          1          1          0  
관악구         0         0         1         2          1          0          2  

[5 rows x 36 columns]

In [31]:
df.columns

Index(['20년1월_살인', '20년2월_살인', '20년3월_살인', '20년4월_살인', '20년5월_살인', '20년6월_살인',
       '20년7월_살인', '20년8월_살인', '20년9월_살인', '20년10월_살인', '20년11월_살인',
       '20년12월_살인', '21년1월_살인', '21년2월_살인', '21년3월_살인', '21년4월_살인', '21년5월_살인',
       '21년6월_살인', '21년7월_살인', '21년8월_살인', '21년9월_살인', '21년10월_살인',
       '21년11월_살인', '21년12월_살인', '22년1월_살인', '22년2월_살인', '22년3월_살인',
       '22년4월_살인', '22년5월_살인', '22년6월_살인', '22년7월_살인', '22년8월_살인', '22년9월_살인',
       '22년10월_살인', '22년11월_살인', '22년12월_살인'],
      dtype='object')

In [32]:
pattern = ['20년', '21년', '22년']
months = [f'{month}월' for month in range(1, 13)]
sorted_columns = [f'{year}{month}_살인' for year in pattern for month in months]

# 패턴에 따라 칼럼 이름 정렬
df = df.reindex(columns=sorted_columns)

In [33]:
df.head()

20년1월_살인  20년2월_살인  20년3월_살인  20년4월_살인  20년5월_살인  20년6월_살인  20년7월_살인  \
자치구                                                                         
강남구         0         0         1         0         0         0         1   
강동구         0         0         0         0         1         0         0   
강북구         0         1         0         0         1         3         1   
강서구         1         0         1         2         3         0         0   
관악구         1         1         0         1         0         2         1   

     20년8월_살인  20년9월_살인  20년10월_살인  ...  22년3월_살인  22년4월_살인  22년5월_살인  \
자치구                                 ...                                 
강남구         0         1          2  ...         1         0         2   
강동구         0         1          1  ...         0         0         0   
강북구         2         1          1  ...         0         1         0   
강서구         0         0          2  ...         0         0         0   
관악구         3         0          0  ...         0         1         2   

     22년6월_살인  22년7월_살인  22년8월_살인  22년9월_살인  22년10월_살인  22년11월_살인  22년12월_살인  
자치구                                                                           
강남구         3         0         1         3          1          1          0  
강동구         1         0         0         1          0          1          1  
강북구         0         2         0         2          0          0          0  
강서구         0         1         0         0          1          1          0  
관악구         0         0         1         2          1          0          2  

[5 rows x 36 columns]

In [34]:
data = df.T
data.to_csv('시계열분석_전처리_살인.csv', encoding='cp949')
data.head()

자치구       강남구  강동구  강북구  강서구  관악구  광진구  구로구  금천구  노원구  도봉구  ...  성동구  성북구  \
20년1월_살인    0    0    0    1    1    0    0    0    1    0  ...    0    0   
20년2월_살인    0    0    1    0    1    0    0    1    0    0  ...    0    0   
20년3월_살인    1    0    0    1    0    0    3    1    1    0  ...    0    0   
20년4월_살인    0    0    0    2    1    0    0    1    0    0  ...    2    0   
20년5월_살인    0    1    1    3    0    0    0    0    0    0  ...    0    0   

자치구       송파구  양천구  영등포구  용산구  은평구  종로구  중구  중랑구  
20년1월_살인    2    0     1    1    1    2   0    1  
20년2월_살인    0    0     1    0    0    3   0    0  
20년3월_살인    0    0     0    0    0    1   0    2  
20년4월_살인    0    1     1    0    0    1   0    3  
20년5월_살인    1    1     1    0    0    0   1    0  

[5 rows x 25 columns]

In [140]:
df = pd.read_csv('시계열분석_전처리_살인.csv', encoding='cp949')
df.head()

Date  강남구  강동구  강북구  강서구  관악구  광진구  구로구  금천구  노원구  ...  성동구  성북구  송파구  \
0  20년1월_살인    0    0    0    1    1    0    0    0    1  ...    0    0    2   
1  20년2월_살인    0    0    1    0    1    0    0    1    0  ...    0    0    0   
2  20년3월_살인    1    0    0    1    0    0    3    1    1  ...    0    0    0   
3  20년4월_살인    0    0    0    2    1    0    0    1    0  ...    2    0    0   
4  20년5월_살인    0    1    1    3    0    0    0    0    0  ...    0    0    1   

   양천구  영등포구  용산구  은평구  종로구  중구  중랑구  
0    0     1    1    1    2   0    1  
1    0     1    0    0    3   0    0  
2    0     0    0    0    1   0    2  
3    1     1    0    0    1   0    3  
4    1     1    0    0    0   1    0  

[5 rows x 26 columns]

In [141]:
# '범죄건수' 문자열 삭제
df['Date'] = df['Date'].str.replace('_살인', '')

df['Date'] = df['Date'].str.replace(r'(\d+)년(\d+)월', r'20\1\2', regex=True)

df['Date'] = pd.to_datetime(df['Date'], format='%Y%m').dt.strftime('%Y%m')

df.head()

Date  강남구  강동구  강북구  강서구  관악구  광진구  구로구  금천구  노원구  ...  성동구  성북구  송파구  \
0  202001    0    0    0    1    1    0    0    0    1  ...    0    0    2   
1  202002    0    0    1    0    1    0    0    1    0  ...    0    0    0   
2  202003    1    0    0    1    0    0    3    1    1  ...    0    0    0   
3  202004    0    0    0    2    1    0    0    1    0  ...    2    0    0   
4  202005    0    1    1    3    0    0    0    0    0  ...    0    0    1   

   양천구  영등포구  용산구  은평구  종로구  중구  중랑구  
0    0     1    1    1    2   0    1  
1    0     1    0    0    3   0    0  
2    0     0    0    0    1   0    2  
3    1     1    0    0    1   0    3  
4    1     1    0    0    0   1    0  

[5 rows x 26 columns]

In [142]:
gu_name = ['성북구', 'Date']

last_3year = list()
for i in range(2023, 2027):
    for j in range(1, 13):
        last_3year.append(['%04d-%02d' % (i,j)])
        
        
last_3year = pd.DataFrame(last_3year, columns = ['ds'])
last_3year['ds']= pd.to_datetime(last_3year['ds'])

for i in range(len(df['Date'])):
    date_time_str = df['Date'][i]
    
    date = datetime.datetime.strptime(str(date_time_str), '%Y%m')  #datetime.datetime.strptime(date, "%Y-%m-%d")
    
    df['Date'][i] = date

seongbuk_predict = pd.DataFrame()

for i,name in enumerate(gu_name):
    data = pd.DataFrame()
    data = df[[name,'Date']]
    data.columns = ['y','ds']
    data['ds'] = pd.to_datetime(data['ds'])

    model = Prophet().fit(data)
    print(i,"model_fit")
    forecast = model.predict(last_3year)
    print(i,"model_predict")
    seongbuk_predict = pd.concat([seongbuk_predict, forecast['yhat']], ignore_index = True, axis=1)

00:06:21 - cmdstanpy - INFO - Chain [1] start processing
00:06:22 - cmdstanpy - INFO - Chain [1] done processing


0 model_fit
0 model_predict


TypeError: Invalid object type at position 0

In [143]:
seongbuk_predict = pd.concat([seongbu_predict, forecast['ds']], ignore_index = True, axis=1)

jungrang_predict.columns = ['중랑구', 'Date']

jungrang_predict.to_csv('jungrang_murder_predict.csv', encoding='cp949',index = False)